# Model

## Build predictive model

In [ ]:
# logistic regression
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression(C = 0.0001, penalty = 'l2', random_state = 42)
clf.fit(X_train_tf, y_train)

We can calculate the probability of readmission for each sample with the fitted model


In [ ]:
model = clf
y_train_preds = model.predict_proba(X_train_tf)[:,1]
y_valid_preds = model.predict_proba(X_valid_tf)[:,1]

## Assess the quality of your model

In [ ]:
# AUC
# acc
# recall
# f1
# precision

# specificity
# prevalence

## Improving the model

We made many choices (a few below) which we could change and see if there is an improvement:
- should we spend time getting more data?
- how to tokenize — should we use stemming?
- how to vectorize — should we change the number of words?
- how to regularized the logistic regression — should we change C or penalty?
- which model to use?

In [ ]:
# For NLP projects that use BOW+logistic regression,
## we can plot the most important words to see if we can gain any insight.

In [ ]:
more_stop_words = ['should','if','it','been','who','during','x']

Some simple things that we can do is try to see the effect of some of our hyperparameters `(max_features and C)`. 

We could run a grid search, but since we only have 2 parameters here we could look at them separately and see the effect.

In [ ]:
from sklearn.model_selection import GridSearchCV
grid_LR = GridSearchCV()

You could try a few other things
- change sub-sample to over-sample
- add stemming or lemmatization to the tokenizer
- test a few different sci-kit learn models
- concatenate all the notes instead of the last discharge summary
- try a deep learning method such as LSTMs
- review the discharge summaries that you are getting wrong

## Finalizing and testing

In [ ]:
rows_not_death = df_adm_notes_clean.DEATHTIME.isnull()
df_adm_notes_not_death = df_adm_notes_clean.loc[rows_not_death].copy()
df_adm_notes_not_death = df_adm_notes_not_death.sample(n = len(df_adm_notes_not_death), random_state = 42)
df_adm_notes_not_death = df_adm_notes_not_death.reset_index(drop = True)

# Save 30% of the data as validation and test data 
df_valid_test=df_adm_notes_not_death.sample(frac=0.30,random_state=42)
df_test = df_valid_test.sample(frac = 0.5, random_state = 42)
df_valid = df_valid_test.drop(df_test.index)

# use the rest of the data as training data
df_train_all=df_adm_notes_not_death.drop(df_valid_test.index)
assert len(df_adm_notes_not_death) == (len(df_test)+len(df_valid)+len(df_train_all)),'math didnt work'

# split the training data into positive and negative
rows_pos = df_train_all.OUTPUT_LABEL == 1
df_train_pos = df_train_all.loc[rows_pos]
df_train_neg = df_train_all.loc[~rows_pos]

# merge the balanced data
df_train = pd.concat([df_train_pos, df_train_neg.sample(n = len(df_train_pos), random_state = 42)],axis = 0)

# shuffle the order of training samples 
df_train = df_train.sample(n = len(df_train), random_state = 42).reset_index(drop = True)

# preprocess the text to deal with known issues
df_train = preprocess_text(df_train)
df_valid = preprocess_text(df_valid)
df_test = preprocess_text(df_test)
my_new_stop_words = ['the','and','to','of','was','with','a','on','in','for','name',              'is','patient','s','he','at','as','or','one','she','his','her','am',                 'were','you','pt','pm','by','be','had','your','this','date',                'from','there','an','that','p','are','have','has','h','but','o',                'namepattern','which','every','also','should','if','it','been','who','during', 'x']
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(lowercase = True, max_features = 3000, 
                       tokenizer = tokenizer_better,
                      stop_words = my_new_stop_words)

# fit the vectorizer
vect.fit(df_train.TEXT.values)
X_train_tf = vect.transform(df_train.TEXT.values)
X_valid_tf = vect.transform(df_valid.TEXT.values)
X_test_tf = vect.transform(df_test.TEXT.values)
y_train = df_train.OUTPUT_LABEL
y_valid = df_valid.OUTPUT_LABEL
y_test = df_test.OUTPUT_LABEL
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression(C = 0.0001, penalty = 'l2', random_state = 42)
clf.fit(X_train_tf, y_train)
model = clf
y_train_preds = model.predict_proba(X_train_tf)[:,1]
y_valid_preds = model.predict_proba(X_valid_tf)[:,1]
y_test_preds = model.predict_proba(X_test_tf)[:,1]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, target_names=target_names))